In [1]:
# Program streamlines orders

# LAST-DAY PUTS section is for assembling orders for the last day

# STATUS: Incomplete

In [3]:
import pandas as pd

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'

In [6]:
df = pd.read_pickle(datapath+'df_main.pkl')

In [ ]:
puts_df = df.loc[(df.Type == 'p') & (df.Delta == 1) & (df.RoM > 0), :]

#****    Logic for last-day expiry puts   ****
#_____________________________________________

put_cols = ['ibSymbol', 'undPrice', 'Strike', 'Expiry', 'DTE', 'pLTP', 'pBidPrice', 
            'Type', 'Mlot', 'Delta', 'RoM']

# Filter out 1-day expiry puts
t_puts = puts_df.loc[puts_df.DTE == 1, put_cols].sort_values(['DTE', 'RoM'], ascending=False)

# Compute price variance percentage
t_puts['price_var'] = (t_puts.pBidPrice - t_puts.pLTP)
t_puts['price_var_pct'] = (t_puts.pBidPrice - t_puts.pLTP)/t_puts.pBidPrice

# Establish target price
# ... Use BidPrice + 0.05 for price variance < -10, else use Last Traded Price
# ... For others use BidPrice + 0.15

t_puts['tgt_price'] = np.where(t_puts.price_var < -10, t_puts.pBidPrice+0.05, t_puts.pLTP)
t_puts.loc[t_puts.tgt_price.isna(), 'tgt_price'] = t_puts.pBidPrice + 0.05